In [ ]:
import numpy as np

import importlib

In [ ]:
def diffuse_cuts(mc):
    """Applies diffuse dataset cuts on a given monte-carlo data.

    Parameters
    ----------
    mc : str | numpy record ndarray
        Monte-carlo data.

    Returns
    -------
    mc_dc : numpy record ndarray
        Monte-carlo data after diffuse dataset cuts.
    """
    mc_dc = mc[(mc['trueDec'] > np.radians(-5)) &
               (np.log10(mc['trueE']) < 8.0) &
               (mc['sigmaOK'] == 0)]
    return mc_dc

In [ ]:
plotter_wkde = importlib.import_module('plotter_wkde_2')

#gammas = np.concatenate(([1.5], np.linspace(2.0, 3.0, 11)))
gammas = [2.0]

logE_list = [2.5]
sigma_p_list = [0.5]

base_path = '/home/ge56lag'

mc = np.load('/{base_path}/Data/diffuse_northern_tracks_MC_KDE/version-001-p00/dataset_8yr_fit_IC86_2012_16_MC_2017_09_29_more_fields.npy'.format(base_path=base_path))

for gamma in gammas:
    spatial_KDE_path = '{}/Software/KDE/output/sig_psi_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_psi_E.pkl'.format(base_path, gamma)
    norm_KDE_path = '{}/Software/KDE/output/sig_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_E.pkl'.format(base_path, gamma)

    spatial_pdf = plotter_wkde.setup(base_path, spatial_KDE_path, norm_KDE_path)

    for logE, sigma_p in zip(logE_list, sigma_p_list):
        plotter_wkde.make_plot(spatial_pdf, mc, logE, sigma_p, gamma=gamma, delta_sigma=0.2, show_quantile=False, output_dir='output_Hans/RGI')

### Hans part

In [ ]:
plotter_wkde = importlib.import_module('plotter_wkde_photospline')

gammas = [1.5, 2.0, 2.5, 3.0]

xpts = np.array([2.5 + i*0.5 for i in range(9)])
ypts = np.array([1.4, 1.1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.4])
tymin = 0.1

base_path = '/home/ge56lag'
output_dir = 'output_Hans/diffuse_cut/photospline'

mc = np.load('/{base_path}/Data/diffuse_northern_tracks_MC_KDE/version-001-p00/dataset_8yr_fit_IC86_2012_16_MC_2017_09_29_more_fields.npy'.format(base_path=base_path))

mc = diffuse_cuts(mc)

for gamma in gammas:
    spatial_KDE_path = '{}/Software/KDE/output/sig_psi_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_psi_E.pkl'.format(base_path, gamma)
    norm_KDE_path = '{}/Software/KDE/output/sig_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_E.pkl'.format(base_path, gamma)

    spatial_pdf = plotter_wkde.setup(base_path, spatial_KDE_path, norm_KDE_path)

    for obj in zip(xpts, ypts):
        logE, tymax = obj
        for sigma_p in np.linspace(tymin, tymax, 10):
            try:
                plotter_wkde.make_plot(spatial_pdf, mc, logE, sigma_p, gamma=gamma, delta_sigma=0.2, show_quantile=False, output_dir=output_dir)
            except ZeroDivisionError:
                print("ZeroDivisionError for:")
                print(gamma, logE, sigma_p)